# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)
    #print("total files: ", len(file_path_list))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line) #get each line in CSV file
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True) # reference :https://www.opentechguides.com/how-to/article/python/182/csv-dialects.html

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue #artist must not be null
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
# Create keyspace named udacity
try:
    session.execute("""
        create keyspace if not exists udacity with replication = {'class': 'SimpleStrategy', 'replication_factor': 1}
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
# set session with above keyspace udacity
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

## ------------------------------------------------- Answer Section --------------------------------------------------

### Question 1 Section: get artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In [8]:
# Creat table song_info.
# The primary key for song_info will be both sessionId and itemInSession, since there could be a chance of multi duplicate
#  sessionId, but when combine with itemInSession, our primary key will be a unique value.

create_table_query1 = "create table if not exists song_info "
create_table_query1 += "(sessionId int, itemInSession int, artist text, song text, length decimal, primary key(sessionID, itemInSession))"

try:
    session.execute(create_table_query1)
except Exception as e:
    print(e)

In [9]:
# Get the CSV file
file = 'event_datafile_new.csv'

# Iterate every line in the CSV file and parse the corresponding data into song_info table
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        insert_table_query1 = "insert into song_info(sessionId, itemInSession, artist, song, length) "
        insert_table_query1 += "values (%s, %s, %s, %s, %s)"
        
        # Transform datatype of data
        artist = line[0]
        first_name = line[1]
        gender = line[2]
        itemInSession = int(line[3])
        last_name = line[4]
        length = float(line[5])
        level = line[6]
        location = line[7]
        sessionId = int(line[8])
        song = line[9]
        userId = str(line[10])
        
        # Creat insert list variable:
        insert_value1 = (sessionId, itemInSession, artist, song, length)

        session.execute(insert_table_query1, (sessionId, itemInSession, artist, song, length))

In [10]:
# Question 1's answer: get the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
# The query consisted of select part which return the information we need like artist name, song name and its length,
#  meanwhile the where part indicate that the data we want to see is from session id 338 at item in session 4.

task_1_query = "select artist, song, length from song_info where sessionId = 338 and itemInSession = 4"

try:
    rows = session.execute(task_1_query)
except Exception as e:
    print(e)
    
for row in rows:
    print (str(row.artist) + ' | ' + str(row.song) + ' | ' +  str(row.length))

Faithless | Music Matters (Mark Knight Dub) | 495.3073


### Question 2 Section: get name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [11]:
# Creat table song_session_listener.
# The primary key for song_session_listener will be both userId and sessionId, which gurantee we have a unique primary key.
# The itemSession will be the table clustering column, to sort data, in this case is in a alphabetical order.

create_table_query2 = "create table if not exists song_session_listener "
create_table_query2 += "(userId int, sessionId int, itemInSession int, artist text, song text, user_name text, primary key((userId, sessionID), itemInSession))"

try:
    session.execute(create_table_query2)
except Exception as e:
    print(e)

In [12]:
# Get the CSV file
file = 'event_datafile_new.csv'

# Iterate every line in the CSV file and parse the corresponding data into song_session_listener table
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        insert_table_query2 = "insert into song_session_listener(userId, sessionId, itemInSession, artist, song, user_name) "
        insert_table_query2 += "values (%s, %s, %s, %s, %s, %s)"

        # Transform datatype of data
        artist = line[0]
        first_name = line[1]
        gender = line[2]
        itemInSession = int(line[3])
        last_name = line[4]
        length = float(line[5])
        level = line[6]
        location = line[7]
        sessionId = int(line[8])
        song = line[9]
        userId = int(line[10])
        
        # Combine last name and first name to create a full name.
        user_name = first_name + ' ' + last_name
        
        # Creat insert list variable:
        insert_value2 = (userId, sessionId, itemInSession, artist, song, user_name)

        session.execute(insert_table_query2, insert_value2)

In [13]:
# Question 2's answer: Get name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
# The query consisted of select part which return the information we need like artist name, song name and its listener,
#  meanwhile the where part indicate that the data we want to see is from user id 10 at session 182.
#  noted that the queried result will be sorted in alphabetical order by itemInSession.

task_2_query = "select artist, song, user_name from song_session_listener where userId = 10 and sessionId = 182"

try:
    rows = session.execute(task_2_query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.artist + ' | ' + row.song + ' | ' + row.user_name)

Down To The Bone | Keep On Keepin' On | Sylvie Cruz
Three Drives | Greece 2000 | Sylvie Cruz
Sebastien Tellier | Kilometer | Sylvie Cruz
Lonnie Gordon | Catch You Baby (Steve Pitron & Max Sanna Radio Edit) | Sylvie Cruz


### Question 3 Section: get every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [14]:
# Creat table song_name_listener.
# The primary key for song_name_listener will be both song and userId, which gurantee we have a unique primary key.
#  If we just set song as primary key, the only (last) one data will be return, since the primary key is not unique

create_table_query3 = "create table if not exists song_name_listener "
create_table_query3 += "(song text, userId int, user_name text, primary key(song, userId))"

try:
    session.execute(create_table_query3)
except Exception as e:
    print(e)            

In [15]:
# Get the CSV file
file = 'event_datafile_new.csv'

# Iterate every line in the CSV file and parse the corresponding data into song_name_listener table
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        insert_table_query3 = "insert into song_name_listener(song, userId, user_name) "
        insert_table_query3 += "values (%s, %s, %s)"

        # Transform datatype of data
        artist = line[0]
        first_name = line[1]
        gender = line[2]
        itemInSession = int(line[3])
        last_name = line[4]
        length = float(line[5])
        level = line[6]
        location = line[7]
        sessionId = int(line[8])
        song = line[9]
        userId = int(line[10])
        
        # Combine last name and first name to create a full name.
        user_name = first_name + ' ' + last_name
        
        # Creat insert list variable:
        insert_value3 = (song, userId, user_name)

        session.execute(insert_table_query3, insert_value3)

In [16]:
# Question 3's answer: get every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
# The query consisted of select part which return the information we need: listener name (or user name),
#  meanwhile the where part indicate that the song that listener listen to is called 'All Hands Against His Own'.

task_3_query = "select user_name from song_name_listener where song = 'All Hands Against His Own'"

try:
    rows = session.execute(task_3_query)
except Exception as e:
    print(e)
    
for row in rows:
    print (row.user_name)


Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [17]:
drop_table_query1 = "drop table song_info"

drop_table_query2 = "drop table song_session_listener"

drop_table_query3 = "drop table song_name_listener"

try:
    rows = session.execute(drop_table_query1)
except Exception as e:
    print(e)
    
try:
    rows = session.execute(drop_table_query2)
except Exception as e:
    print(e)

try:
    rows = session.execute(drop_table_query3)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [18]:
session.shutdown()
cluster.shutdown()

In [19]:
# This is the end of this notebook